In [1]:
#r "BoSSSpad.dll"
#r "SAIDT.dll"
#r "ApplicationWithIDT.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using ApplicationWithIDT;
using SAIDT;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [2]:
string dbPath=null;

load a control file and the solver (but do not run a simulation)

In [3]:
using SAIDT;
BoSSS.Solution.Application.InitMPI();
BoSSS.Solution.Application.DeleteOldPlotFiles();
var pSAIDT = new SAIDTMain();
var CSAIDT = SAIDTHardCodedControl.StraightShock(
                    dbPath: dbPath,
                    MaxIterations: 50,
                    dgDegree: 4,
                    numOfCellsX: 3,
                    numOfCellsY: 3,
                    OptiNumOfCellsX: 3,
                    OptiNumOfCellsY: 3,
                    agg: 0.4,
                    ImmediatePlotPeriod: -1,
                    optiLevelSetType: OptiLevelSetType.SinglePhaseField,
                    LSDegree: 3
                    );
CSAIDT.SessionName="SAIDT_DGLevelSet";
CSAIDT.ProjectName="SAIDT_DGLevelSet";
pSAIDT.Init(CSAIDT);
pSAIDT.InitializeEverything();


rm;
Session ID: 00000000-0000-0000-0000-000000000000, DB path: 'EMPTY'
Session directory 'EMPTY\sessions\00000000-0000-0000-0000-000000000000'.
Grid repartitioning method: METIS
Grid repartitioning options: 
Number of cell Weights: 0


*****************************************************************************************
*  X   X   DDD   GGG   -   III  SSS  TTTTT  *
*   X X    D  D G      -    I  S      T     *
*    X     D  D G  GG  -    I   SSS   T     *      XDG Implicit Shock Tracking Solver    
*   X X    D  D G   G  -    I      S  T     *
*  X   X   DDD   GGG   -   III  SSS   T     *
*****************************************************************************************
*  
*  solver:                             SAIDT.SAIDTMain
*  project:                            SAIDT_DGLevelSet
*  session:                            SAIDT_DGLevelSet
*  optimization problem:               FullEnRes
*  merit function:                     L1Merit
*  level set type:                     Sing

function to project on level set

In [5]:
Func<double[],double> discCircA= delegate(double[] X){
    var y=X[1];
    var x=X[0];
    double result=0.0;
    if (x>1.0/3.0 &&x<2.0/3.0 &&y>1.0/3.0 &&y<2.0/3.0 )
    {
        // Apply the first function for y > 0.5
        result = x>0.5 ? 1+1*Math.Sin(100*x)*Math.Sin(100*y): 1+1*Math.Cos(100*x)*Math.Cos(100*y);
    }
    else 
    {
        // Apply the second function for y < 0.75
        result = x+y;
    }
    
    return result;
};
Func<double[],double> discCircB= delegate(double[] X){
    var y=X[1];
    var x=X[0];
    double result=0.0;
    if (x>1.0/3.0 &&x<2.0/3.0 &&y>1.0/3.0 &&y<2.0/3.0 )
    {
        // Apply the first function for y > 0.5
        result = x< 0.5 ? 4+ 1*Math.Sin(100*x)*Math.Sin(100*y): 4+1*Math.Cos(100*x)*Math.Cos(100*y);
    }
    else 
    {
        // Apply the second function for y < 0.75
        result = 4;
    }
    
    return result;
};

plotting function

In [6]:
using BoSSS.Solution.Tecplot;
using System.IO;
Directory.GetFiles(".", "*_SP_*").ForEach(file => File.Delete(file));

public void plotSF(string name){
    List<DGField> flds= new List<DGField>();
foreach(var field in pSAIDT.RegisteredFields){
    if(field is XDGField xfield){
        flds.Add(xfield.GetSpeciesShadowField("L"));
        flds.Add(xfield.GetSpeciesShadowField("R"));
        flds.Add(field);
    }else{
        flds.Add(field);
    }
}
var texplot = new Tecplot(flds[0].GridDat, 5);
texplot.PlotFields( name,0.0,flds);
}

project function, plot, do reintialization, plot again

In [7]:
BoSSS.Solution.Application.DeleteOldPlotFiles();
pSAIDT.ConservativeFields[0].GetSpeciesShadowField("L").ProjectField(discCircA);
pSAIDT.ConservativeFields[0].GetSpeciesShadowField("R").ProjectField(discCircB);
pSAIDT.Control.ApplyReiInit=true;
pSAIDT.GetPerssonSensor(false);
pSAIDT.Control.reInitTols=new double[] {-2,-2,-2,-2,-2,-2};
plotSF("reinit_0");
pSAIDT.Reinitialization();
pSAIDT.GetPerssonSensor(false);
plotSF("reinit_1");

rm SAIDT_DGLevelSet_0.plt;
Reinitialized Cells:(4 L) ,(4 R) , 
